In [ ]:
from utils import ChatOpenAI
from getpass import getpass

#course_api_key= "Введите ваш API ключ с курса"
course_api_key = getpass(prompt='Введите API ключ')

# Инициализируем языковую модель
llm = ChatOpenAI(course_api_key=course_api_key)

In [1]:
import re
import pandas as pd
from tqdm import tqdm
from getpass import getpass

from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain, TransformChain
from langchain.output_parsers import ResponseSchema, StructuredOutputParser

df = pd.read_csv("https://stepik.org/media/attachments/lesson/1110883/raw_texts.csv")
df.head()

,raw_text
0,"The sun was setting, casting long shadows over..."
1,"Le soleil se couchait, jetant de longues ombre..."
2,"El sol se estaba poniendo, proyectando largas ..."
3,"La ciudad estaba llena de vida, sus calles lle..."
4,"La ville était pleine de vie, ses rues remplie..."


In [ ]:
def clean_text(inputs: dict) -> dict:
    text = inputs["text"]

    pattern = r'[¿¡£]'
    text =  re.sub(pattern, '', text)
   
    
    return {"text": text}

In [ ]:
def clean_text_nodict(text: str) -> dict:

    pattern = r'[¿¡£]'
    text =  re.sub(pattern, '', text)
   
    
    return text

In [ ]:
# Определим схемы ответа 
language_schema = gift_schema = ResponseSchema(name="language",
                             description="на каком языке написано произведение")

person_schema = ResponseSchema(name="person",
                             description="имя главного героя")

response_schemas = [language_schema, person_schema]
output_parser = StructuredOutputParser.from_response_schemas(response_schemas)
format_instructions = output_parser.get_format_instructions()
# print(format_instructions)
# output_dict = output_parser.parse(response.content)

In [ ]:
# создадим шаблон и промпт
template = '''Из следующего текста извлеки информацию:
1. Язык на котором написан текст. Указать на английском языке (например, не Русский, а Russian. Не Deutsch, а German)
2. Главный герой рассказа. Имя главного героя произведения. На том же языке, как оно указано в тексте. (Hans, Pierrot, Анна)
text: {text}

{format_instructions}
Результат:'''

# Отличие между input_variables и partial_variables в LangChain заключается в том, что они представляют разные типы переменных:
# input_variables — это список имён переменных, которые нужно предоставить во время выполнения, чтобы заполнить заполнители в шаблоне подсказки 1
# partial_variables — это словарь, где ключи — имена переменных, которые имеют фиксированное значение и не нужно предоставлять во время выполнения, а значения — эти фиксированные значения. 
# Таким образом, input_variables указывают, какие переменные нужно заполнять во время выполнения, а partial_variables — это переменные с фиксированными значениями, которые используются при генерации подсказки. 

prompt = PromptTemplate(
    template=template,
    input_variables=["text"],
    partial_variables={"format_instructions": format_instructions},
)


print(prompt.template)

In [ ]:
clean_text_chain = clean_text | prompt | llm 


In [ ]:
df_2 = pd.DataFrame()  #columns=['text','language','person']
text_list = []
language_list = []
person_list = []
for text in tqdm(df['raw_text']):
    
    result = clean_text_chain.invoke({'text': text,'format_instructions': format_instructions })
    output_dict = output_parser.parse(result.content)
    text_list.append(clean_text_nodict(text))
    language_list.append(output_dict.get('language'))
    person_list.append(output_dict.get('person'))
    # break
df_2['text'] = text_list
df_2['language'] = language_list
df_2['main_character'] = person_list
df_2

In [ ]:
df_2.to_csv('3.2.9_solution.csv', index=False)

In [ ]:
import re
def del_additional_signs(string: str) -> str:
    result = re.sub(r'[!?]+', lambda m: '!' if '!' in m.group() else '?', string) # Заменяем все последовательности на один знак
    result = re.sub(r'^[!?]+', '', result) # Удаляем все знаки препинания в начале строки
    
    return result

# не изменяйте фрагмент кода ниже
string = input()
print(del_additional_signs(string))
